In [ ]:
# -*- coding: utf-8 -*-
"""Deepclassifier-aid1239_revised.ipynb"""

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from google.colab import drive


In [ ]:
# Ensure reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load the dataset
#df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/AID1239_extracted_descriptor_data.csv')
#/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv')
df_copy = df.copy()


In [ ]:
df.shape

(4000, 24283)

In [ ]:
# EDA 1
# Calculate the percentage of NaN values in each column


nan_percentage = df.isna().mean() * 100

# Find columns where more than 50% of the values are NaN
columns_with_nan_above_50 = nan_percentage[nan_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% NaN values: {len(columns_with_nan_above_50)}")
print("Columns:", columns_with_nan_above_50)


Number of columns with more than 50% NaN values: 8416
Columns: ['morgan_counts_331095558', 'morgan_counts_670649866', 'morgan_counts_4104655381', 'morgan_counts_3310046752', 'morgan_counts_810097194', 'morgan_counts_729008687', 'morgan_counts_2995670577', 'morgan_counts_1279384625', 'morgan_counts_616215095', 'morgan_counts_3183596602', 'morgan_counts_2742604860', 'morgan_counts_3703709255', 'morgan_counts_3867751498', 'morgan_counts_2190227530', 'morgan_counts_270573645', 'morgan_counts_672648274', 'morgan_counts_4018573394', 'morgan_counts_1766270060', 'morgan_counts_3020277366', 'morgan_counts_670700536', 'morgan_counts_4021193861', 'morgan_counts_752412296', 'morgan_counts_3660960393', 'morgan_counts_4173243026', 'morgan_counts_4019697815', 'morgan_counts_3766531226', 'morgan_counts_810098337', 'morgan_counts_4263347881', 'morgan_counts_296409257', 'morgan_counts_178375861', 'morgan_counts_2641106110', 'morgan_counts_318115522', 'morgan_counts_1496549075', 'morgan_counts_3020085465

In [ ]:
# EDA 2
# Calculate the percentage of zero values in each column



zero_percentage = (df == 0).mean() * 100

# Find columns where more than 50% of the values are 0
columns_with_zero_above_50 = zero_percentage[zero_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% zero values: {len(columns_with_zero_above_50)}")
print("Columns:", columns_with_zero_above_50)


Number of columns with more than 50% zero values: 3336
Columns: ['rdf27', 'rdf28', 'rdf29', 'rdf56', 'rdf57', 'rdf58', 'rdf59', 'rdf86', 'rdf87', 'rdf88', 'rdf89', 'rdf117', 'rdf118', 'rdf119', 'rdf147', 'rdf148', 'rdf149', 'rdf177', 'rdf178', 'rdf179', 'rdf207', 'rdf208', 'rdf209', 'maccs0', 'maccs1', 'maccs2', 'maccs3', 'maccs4', 'maccs5', 'maccs6', 'maccs7', 'maccs8', 'maccs9', 'maccs10', 'maccs11', 'maccs12', 'maccs13', 'maccs14', 'maccs15', 'maccs16', 'maccs17', 'maccs18', 'maccs19', 'maccs20', 'maccs21', 'maccs22', 'maccs23', 'maccs24', 'maccs25', 'maccs26', 'maccs27', 'maccs28', 'maccs29', 'maccs30', 'maccs31', 'maccs32', 'maccs33', 'maccs34', 'maccs35', 'maccs36', 'maccs37', 'maccs38', 'maccs39', 'maccs40', 'maccs41', 'maccs42', 'maccs43', 'maccs44', 'maccs45', 'maccs46', 'maccs47', 'maccs48', 'maccs49', 'maccs50', 'maccs51', 'maccs52', 'maccs53', 'maccs54', 'maccs55', 'maccs56', 'maccs57', 'maccs58', 'maccs59', 'maccs60', 'maccs61', 'maccs62', 'maccs63', 'maccs64', 'maccs66', 

In [ ]:
# Remove all the columns which falls in this categories:
# Number of columns with more than 50% NaN values:
# Number of columns with more than 50% zero values:

import pandas as pd

# Assuming 'data' is your DataFrame

# 1. Identify columns with more than 50% NaN values
threshold_nan = len(df) * 0.5
columns_with_nan = df.columns[df.isna().sum() > threshold_nan]

# 2. Identify columns with more than 50% zero values
threshold_zero = len(df) * 0.5
columns_with_zero = df.columns[(df == 0).sum() > threshold_zero]

# 3. Combine both sets of columns to drop
columns_to_drop = set(columns_with_nan).union(set(columns_with_zero))

# 4. Drop these columns from the DataFrame
data_cleaned = df.drop(columns=columns_to_drop)

# Check the shape of the new DataFrame to confirm the columns have been removed
print(f"Original shape: {df.shape}")
print(f"New shape after dropping columns: {data_cleaned.shape}")


Original shape: (4000, 24283)
New shape after dropping columns: (4000, 12531)


In [ ]:
df = data_cleaned.copy()

In [ ]:
# Step 1: Fill null values with zero
df.fillna(0, inplace=True)
df = df.round(3)


In [ ]:
# Save the preprocessed dataset for future reference
df.to_csv('/content/drive/MyDrive/Datasets/AID_1239/preprocessed_AID1239_12531Columns_Active_Inactive.csv', index=False)
print(df.head())

   Unnamed: 0  PUBCHEM_CID  PUBCHEM_SID  \
0           0      1449342     24817956   
1           1      3242114      4247974   
2           2      9594900     17507393   
3           3       823601      7975245   
4           4      3239831      4245352   

                                            SMILES PUBCHEM_ACTIVITY_OUTCOME  \
0  C1=CC=C(C=C1)C(=O)NC2=NC=C(C=C2)NC(=O)C3=CC=CS3                   Active   
1          C1COCCN1C(=O)C2=NOC(=C2)C3=CC=C(C=C3)Cl                   Active   
2      CC1=CC(=NC2=CC=CC=C12)N/N=C/C3=CC(=CC=C3)Br                   Active   
3            CC1=C(C=CO1)C(=O)NC2=CC3=CC=CC=C3C=C2                   Active   
4            C1=CC=C(C(=C1)NC(=O)C2=CC=C(C=C2)Cl)O                   Active   

                                        SMILES.1 MOLECULEID  autocorr2d0  \
0  O=C(NC1=CN=C(NC(=O)C2=CC=CC=C2)C=C1)C1=CC=CS1   M3503957        3.470   
1        ClC1=CC=C(C=C1)C1=CC(=NO1)C(=O)N1CCOCC1   M2730310        3.353   
2    CC1=CC(N\N=C\C2=CC(Br)=CC=C2)=NC2

In [ ]:
df.head()

,Unnamed: 0,PUBCHEM_CID,PUBCHEM_SID,SMILES,PUBCHEM_ACTIVITY_OUTCOME,SMILES.1,MOLECULEID,autocorr2d0,autocorr2d1,autocorr2d2,...,"atom_pairs_((N,1,2),14,(*,1,0))","atom_pairs_((N,3,0),12,(O,1,1))",morgan_counts_943520092,"atom_pairs_((O,2,0),6,(S,4,0))",morgan_counts_640577968,"atom_pairs_((C,2,2),14,(*,1,0))","bpf_((B,5,0),16,(B,5,0))","atom_pairs_((C,4,0),15,(Cl,1,0))",morgan_counts_469020719,morgan_counts_594640005
0,0,1449342,24817956,C1=CC=C(C=C1)C(=O)NC2=NC=C(C=C2)NC(=O)C3=CC=CS3,Active,O=C(NC1=CN=C(NC(=O)C2=CC=CC=C2)C=C1)C1=CC=CS1,M3503957,3.470,3.799,3.750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3242114,4247974,C1COCCN1C(=O)C2=NOC(=C2)C3=CC=C(C=C3)Cl,Active,ClC1=CC=C(C=C1)C1=CC(=NO1)C(=O)N1CCOCC1,M2730310,3.353,3.709,3.697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,9594900,17507393,CC1=CC(=NC2=CC=CC=C12)N/N=C/C3=CC(=CC=C3)Br,Active,CC1=CC(N\N=C\C2=CC(Br)=CC=C2)=NC2=CC=CC=C12,M3349978,3.462,3.850,3.880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,823601,7975245,CC1=C(C=CO1)C(=O)NC2=CC3=CC=CC=C3C=C2,Active,CC1=C(C=CO1)C(=O)NC1=CC2=CC=CC=C2C=C1,M2603287,3.196,3.537,3.501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3239831,4245352,C1=CC=C(C(=C1)NC(=O)C2=CC=C(C=C2)Cl)O,Active,OC1=CC=CC=C1NC(=O)C1=CC=C(Cl)C=C1,M1281287,3.128,3.483,3.514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Define target and columns to drop
target_column = 'PUBCHEM_ACTIVITY_OUTCOME'
drop_columns = ['Unnamed: 0','SMILES.1','PUBCHEM_SID', 'PUBCHEM_CID', 'SMILES', 'MOLECULEID']


In [ ]:
# Drop unnecessary columns
df.drop(columns=drop_columns, inplace=True)


In [ ]:
# Encode target column (Active/Inactive)
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column])

In [ ]:
# Split the features and target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

In [ ]:
# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Check for NaN and infinity values, replace if needed
X_train[np.isinf(X_train)] = np.nan
X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))

X_test[np.isinf(X_test)] = np.nan
X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, callbacks=[early_stopping])


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.6455 - loss: 4.4548 - val_accuracy: 0.7788 - val_loss: 3.5156
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7659 - loss: 3.3761 - val_accuracy: 0.7987 - val_loss: 2.7593
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8290 - loss: 2.5753 - val_accuracy: 0.7950 - val_loss: 2.1734
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8600 - loss: 1.9603 - val_accuracy: 0.8050 - val_loss: 1.8138
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8813 - loss: 1.6092 - val_accuracy: 0.8050 - val_loss: 1.6028
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8898 - loss: 1.4058 - val_accuracy: 0.8200 - val_loss: 1.4967
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9102 - loss: 1.2482 - val_accuracy: 0.8125 - val_loss: 1.4136
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9196 - loss: 1.1415 - val_accuracy: 0.7850 - val_loss

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)


In [ ]:
# Evaluate the model on training data
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)

In [ ]:
# Make predictions on training data
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
# Make predictions
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
# Calculate metrics for training data
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

In [ ]:
# Calculate metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)


In [ ]:
# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("Model Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

Model Evaluation Metrics:
train_accuracy: 0.9656
train_roc_auc: 0.9970
train_f1: 0.9667
train_kappa: 0.9312
train_precision: 0.9489
train_recall: 0.9852
test_accuracy: 0.8112
test_roc_auc: 0.8797
test_f1: 0.8076
test_kappa: 0.6229
test_precision: 0.7789
test_recall: 0.8386


**Below is Code with Optuna**

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping

# Define the Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    n_layers = trial.suggest_int("n_layers", 2, 4)
    units = trial.suggest_int("units", 32, 256, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Build the model
    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    for _ in range(n_layers - 1):
        model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
        units //= 2
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                        batch_size=batch_size, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Calculate additional metrics on test data
    y_pred = model.predict(X_test).flatten()
    y_pred_classes = (y_pred > 0.5).astype(int)

    if len(np.unique(y_test)) < 2:
        test_roc_auc = None
    else:
        test_roc_auc = roc_auc_score(y_test, y_pred)

    test_f1 = f1_score(y_test, y_pred_classes)
    test_kappa = cohen_kappa_score(y_test, y_pred_classes)
    test_precision = precision_score(y_test, y_pred_classes)
    test_recall = recall_score(y_test, y_pred_classes)

    # Return the test accuracy as the objective to maximize
    return test_acc

# Run Optuna for hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Best hyperparameters from Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Build the final model with the best hyperparameters
n_layers = best_params["n_layers"]
units = best_params["units"]
dropout_rate = best_params["dropout_rate"]
l2_reg = best_params["l2_reg"]
learning_rate = best_params["learning_rate"]
batch_size = best_params["batch_size"]

model = Sequential()
model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
model.add(Dropout(dropout_rate))
for _ in range(n_layers - 1):
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    units //= 2
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the final model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                    batch_size=batch_size, callbacks=[early_stopping], verbose=1)

# Evaluate the final model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Train metrics
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

# Test metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


[I 2024-11-17 07:57:02,101] A new study created in memory with name: no-name-47791280-ab13-4c1e-ac5c-c1a02c421cef
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-17 07:57:19,625] Trial 0 finished with value: 0.8199999928474426 and parameters: {'n_layers': 4, 'units': 128, 'dropout_rate': 0.2, 'l2_reg': 0.00613117000369051, 'learning_rate': 3.0445345360974923e-05, 'batch_size': 64}. Best is trial 0 with value: 0.8199999928474426.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-17 07:57:29,379] Trial 1 finished with value: 0.8025000095367432 and parameters: {'n_layers': 2, 'units': 256, 'dropout_rate': 0.5, 'l2_reg': 5.00508760282657e-05, 'learning_rate': 7.603619929208971e-05, 'batch_size': 64}. Best is trial 0 with value: 0.8199999928474426.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


[I 2024-11-17 08:03:23,948] Trial 29 finished with value: 0.8112499713897705 and parameters: {'n_layers': 4, 'units': 128, 'dropout_rate': 0.2, 'l2_reg': 0.00556319846504643, 'learning_rate': 0.00011187754354679655, 'batch_size': 128}. Best is trial 13 with value: 0.824999988079071.


Best hyperparameters: {'n_layers': 4, 'units': 64, 'dropout_rate': 0.30000000000000004, 'l2_reg': 0.008067222122029354, 'learning_rate': 8.194807693408146e-05, 'batch_size': 64}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.5328 - loss: 2.3513 - val_accuracy: 0.6938 - val_loss: 2.1955
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5810 - loss: 2.2211 - val_accuracy: 0.7262 - val_loss: 2.1503
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6124 - loss: 2.1521 - val_accuracy: 0.7513 - val_loss: 2.0999
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6350 - loss: 2.1145 - val_accuracy: 0.7688 - val_loss: 2.0581
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6452 - loss: 2.0769 - val_accuracy: 0.7788 - val_loss: 2.0170
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6493 - loss: 2.0135 - val_accuracy: 0.7875 - val_loss: 1.9684
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6783 - loss: 1.9638 - val_accuracy: 0.7900 - val_loss: 1.9252
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6962 - loss: 1.9226 - val_accuracy: 0.7925 - val_loss

below code with old ones

In [ ]:
# Step 4: Build the model
model = Sequential()


In [ ]:
# Input layer with L2 regularization
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))  # Dropout for regularization


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Hidden layers with L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))

In [ ]:
# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Add callbacks for early stopping and model checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Early stopping to prevent overfitting, model checkpoint to save the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Jyothi Research Paper/best_model_fulldescriptors_aid1239.keras', save_best_only=True)


In [ ]:
# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.6455 - loss: 4.4548 - val_accuracy: 0.7788 - val_loss: 3.5156
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7659 - loss: 3.3761 - val_accuracy: 0.7987 - val_loss: 2.7593
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8290 - loss: 2.5753 - val_accuracy: 0.7950 - val_loss: 2.1734
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8600 - loss: 1.9603 - val_accuracy: 0.8050 - val_loss: 1.8138
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8813 - loss: 1.6092 - val_accuracy: 0.8050 - val_loss: 1.6028
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8898 - loss: 1.4058 - val_accuracy: 0.8200 - val_loss: 1.4967
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9102 - loss: 1.2482 - val_accuracy: 0.8125 - val_loss: 1.4136
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9196 - loss: 1.1415 - val_accuracy: 0.7850 - val_loss

In [ ]:
# Step 6: Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8195 - loss: 0.9831 
Test Accuracy: 81.12%


In [ ]:
# Step 7: Make predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Use 0.5 as threshold for binary classification


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [ ]:
# Assuming you have 'df_copy' (original dataframe) and 'y_pred_classes' available

# Predict on the entire dataset to get predictions for all rows
# (Assuming 'X' contains features for all rows in 'df_copy')
y_pred_all = model.predict(X)
y_pred_classes_all = (y_pred_all > 0.5).astype(int)

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame()

# Assuming 'df_copy' contains 'PUBCHEM_CID' and 'PUBCHEM_ACTIVITY_OUTCOME'
new_df['PUBCHEM_CID'] = df_copy['PUBCHEM_CID']  # Use values from the original df
new_df['PUBCHEM_ACTIVITY_OUTCOME'] = df_copy['PUBCHEM_ACTIVITY_OUTCOME'] # Use values from original df

# Add the predicted classes for all rows and map to "Active"/"Inactive"
new_df['y_pred_classes'] = y_pred_classes_all.flatten()
new_df['y_pred_classes'] = new_df['y_pred_classes'].map({1: 'Active', 0: 'Inactive'}) # Map 1 to 'Active' and 0 to 'Inactive'

# Display or save the new DataFrame as needed
print(new_df.head())
#new_df.to_csv('predicted_outcomes.csv', index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
   PUBCHEM_CID PUBCHEM_ACTIVITY_OUTCOME y_pred_classes
0      1449342                   Active       Inactive
1      3242114                   Active       Inactive
2      9594900                   Active       Inactive
3       823601                   Active       Inactive
4      3239831                   Active       Inactive


In [ ]:
# Step 8: Evaluate the results
compound_ids = df_copy['PUBCHEM_CID'].values
compound_ids_train, compound_ids_test = train_test_split(compound_ids, test_size=0.2, random_state=42)


In [ ]:
results = [
   f"Compound {compound_id}: This is active" if pred == 1 else f"Compound {compound_id}: This is inactive"
   for compound_id, pred in zip(compound_ids_test, y_pred_classes.flatten())
]

results

['Compound 2986465: This is inactive',
 'Compound 1319487: This is active',
 'Compound 2999802: This is inactive',
 'Compound 16190573: This is active',
 'Compound 666181: This is active',
 'Compound 135472859: This is inactive',
 'Compound 711992: This is inactive',
 'Compound 6898097: This is inactive',
 'Compound 2986519: This is inactive',
 'Compound 135490211: This is inactive',
 'Compound 1867778: This is active',
 'Compound 5762397: This is active',
 'Compound 1481967: This is inactive',
 'Compound 2831188: This is active',
 'Compound 2515723: This is active',
 'Compound 4120298: This is active',
 'Compound 1481828: This is inactive',
 'Compound 684082: This is inactive',
 'Compound 5730347: This is inactive',
 'Compound 3237354: This is inactive',
 'Compound 752122: This is inactive',
 'Compound 5721129: This is inactive',
 'Compound 5724809: This is active',
 'Compound 755827: This is inactive',
 'Compound 4248181: This is active',
 'Compound 704968: This is inactive',
 'Compo

In [ ]:
# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[332  90]
 [ 61 317]]


In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))



Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.79      0.81       422
           1       0.78      0.84      0.81       378

    accuracy                           0.81       800
   macro avg       0.81      0.81      0.81       800
weighted avg       0.81      0.81      0.81       800



In [ ]:
# Optional: Save the model for future use
model.save('/content/drive/MyDrive/Jyothi Research Paper/final_deep_classifier_model.keras')
